In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Text_Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [10]:
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

In [9]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [11]:
import evaluate

In [17]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,batch):
        dialogues = [str(x) for x in batch["dialogue"]]
        summaries = [str(x) for x in batch["summary"]]

        model_inputs = self.tokenizer(
            dialogues,
            max_length=1024,
            truncation=True,
            padding="max_length"
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                summaries,
                max_length=128,
                truncation=True,
                padding="max_length"
            )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-08-24 02:03:58,262: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-24 02:03:58,267: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-24 02:03:58,270: INFO: common: created directory at: artifacts_root]
[2025-08-24 02:03:58,270: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Text_Summarizer\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 37688.27 examples/s]
